In [151]:
import tensorflow as tf
from tensorflow.keras import layers,Sequential,datasets
tf.random.set_seed(4653)

In [152]:
cnn_layers = [
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='same'),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='same'),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='same'),
]

In [153]:
model = Sequential(cnn_layers)

In [154]:
model.build(input_shape=[None, 32, 32, 3])
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_76 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 conv2d_77 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_78 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 conv2d_79 (Conv2D)          (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 8, 8, 128)        0         
 g2D)                                                

In [155]:
fc_net = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(100, activation='relu'),
])

In [156]:
fc_net.build(input_shape=[None, 4 * 4 * 256])

In [157]:
def process(x,y):
    x = tf.cast(x, tf.float32) / 255.
    y = tf.cast(y, tf.int32)
    return x,y
(train_x, train_y), (test_x, test_y) = datasets.cifar10.load_data()
train_y = tf.squeeze(train_y)
test_y = tf.squeeze(test_y)
train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_db = train_db.map(process).shuffle(10000).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_db = test_db.map(process).batch(128)

In [158]:
train_db_iter = next(iter(train_db))
train_db_iter[0].shape, train_db_iter[1].shape

(TensorShape([128, 32, 32, 3]), TensorShape([128]))

In [159]:
variables = model.trainable_variables + fc_net.non_trainable_variables
optimizer = tf.keras.optimizers.Adam(0.001)
for epoch in range(1):
    for step, (x, y) in enumerate(train_db):

        with tf.GradientTape() as tape:
            output = model(x)
            output = tf.reshape(output, [-1, 4 * 4 * 256])
            logits = fc_net(output)
            y_onehot = tf.one_hot(y, depth=100)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_onehot,logits,from_logits=True))
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        if step % 100 == 0:
            print(f"Epoch {epoch}, step {step}, loss: {loss}")

Epoch 0, step 0, loss: 4.608943939208984


KeyboardInterrupt: 

In [150]:
total_num,total_cor = 0,0
for x,y in test_db:
    output = model(x)
    output = tf.reshape(output, [-1, 4 * 4 * 256])
    logits = fc_net(output)
    prob = tf.nn.softmax(logits)
    pred = tf.argmax(prob, axis=-1)
    pred = tf.cast(pred, tf.int32)

    cor = tf.cast(tf.equal(pred, y), tf.int32)
    total_cor = total_cor + int(cor)
    total_num = total_num + x.shape[0]
accuracy = total_cor/total_num
print(f"Accuracy: {accuracy}")


TypeError: only length-1 arrays can be converted to Python scalars